In [14]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import csv

train_data = pd.read_csv(r"C:\Users\USER\Python_Projects\Titanic\Datasets\train.csv")
test_data = pd.read_csv(r"C:\Users\USER\Python_Projects\Titanic\Datasets\test.csv")

# 特徴量の選択
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

train_data["Age"] = train_data["Age"].fillna(train_data["Age"].median())
train_data["Embarked"] = train_data["Embarked"].fillna("S")
train_data.loc[train_data["Sex"] == "male", "Sex"] = 0
train_data.loc[train_data["Sex"] == "female", "Sex"] = 1
train_data.loc[train_data["Embarked"] == "S", "Embarked"] = 0
train_data.loc[train_data["Embarked"] == "C", "Embarked"] = 1
train_data.loc[train_data["Embarked"] == "Q", "Embarked"] = 2

test_data["Age"] = test_data["Age"].fillna(test_data["Age"].median())
test_data.loc[test_data["Sex"] == "male", "Sex"] = 0
test_data.loc[test_data["Sex"] == "female", "Sex"] = 1
test_data.loc[test_data["Embarked"] == "S", "Embarked"] = 0
test_data.loc[test_data["Embarked"] == "C", "Embarked"] = 1
test_data.loc[test_data["Embarked"] == "Q", "Embarked"] = 2
test_data.loc[152, "Fare"] = test_data.Fare.median()

train_data = train_data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

test_data = test_data.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis = 1)

train, val = train_test_split(train_data)

class TitanicDataset(Dataset):
    def __init__(self, data, is_test=False):
        if is_test:
            self.X = data[features].values.astype(np.float32)
            self.y = None
        else:
            self.X = data[features].values.astype(np.float32)
            self.y = data['Survived'].values.astype(np.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx], self.y[idx]
        else:
            return self.X[idx]

train_dataset = TitanicDataset(train)
val_dataset = TitanicDataset(val)
test_dataset = TitanicDataset(test_data, is_test=True)

# DataLoaderの定義
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")